# Benchmarks for cold-start

The idea is to compare DPP / random / mosaïque for the selection of the first items to rate for a newcomer.

## Preliminaries: DPP sampling

Load vectors from the saved SVD

In [113]:
from mangaki.utils.data import Dataset
dataset = Dataset()

In [114]:
from mangaki.utils.svd import MangakiSVD
algo = MangakiSVD()
algo.load(algo.get_backup_filename())
dataset.load('ratings-' + algo.get_backup_filename())

In [115]:
import numpy as np
rated_by_jj = np.array(User.objects.get(id=1).rating_set.values_list('work', 'work__title', 'choice'), dtype=[('work_id', 'i2'), ('work_title', 'U128'), ('choice', 'S8')])

In [132]:
rated_works = rated_by_jj['work_id']
encoded_work_ids = dataset.encode_works(rated_works)

In [117]:
vectors = algo.VT.T[encoded_work_ids]
L = vectors.dot(vectors.T)

In [118]:
D, V = np.linalg.eig(L.T)
D = np.real(D)
V = np.real(V)

If we want to summarize JJ's profile, which ratings should we keep?

In [195]:
from mangaki.utils import dpplib
sampled_indices = list(map(int, dpplib.sample_k(5, D, V)))
print(rated_by_jj[sampled_indices])
dpp_works = rated_works[sampled_indices]

def get_dpp_works(k):
    sampled_indices = list(map(int, dpplib.sample_k(k, D, V)))
    return rated_works[sampled_indices]

import random
def get_random_works(k):
    return np.array(random.sample(list(rated_works), k))

[(  13, 'Code Geass: Hangyaku no Lelouch R2', b'like')
 ( 260, 'Pokemon 3: The Movie', b'willsee')
 ( 410, 'Porco Rosso', b'favorite')
 ( 110, 'Suzumiya Haruhi no Shoushitsu', b'favorite')
 (2395, 'Umineko: When They Cry', b'willsee')]


## Simulation

In [197]:
print(Rating.objects.filter(user_id=1).count(), 'ratings for JJ')
ds = Dataset()
anonymized = ds.make_anonymous_data(Rating.objects.all())

536 ratings for JJ


In [198]:
def select_works_to_rate():
    if METHOD == 'dpp':
        kept_works = get_dpp_works(NB_WORKS)
    else:
        kept_works = get_random_works(NB_WORKS)
    return kept_works

In [199]:
def split_train_test(kept_works):
    i_train = []
    i_test = []
    my_encoded_user_id = ds.encode_user[1]
    encoded_kept_works = set(ds.encode_works(kept_works))
    for i, (encoded_user_id, encoded_work_id) in enumerate(anonymized.X):
        if encoded_user_id == my_encoded_user_id and encoded_work_id not in encoded_kept_works:
            i_test.append(i)
        else:
            i_train.append(i)
    return i_train, i_test

In [207]:
from mangaki.utils.knn import MangakiKNN
from sklearn.metrics import mean_squared_error
def compute_error(i_train, i_test):
    knn = MangakiKNN()
    knn.set_parameters(anonymized.nb_users, anonymized.nb_works)
    knn.fit(anonymized.X[i_train], anonymized.y[i_train])
    y_pred = knn.predict(anonymized.X[i_test])
    print('predicted:', y_pred[:5])
    print('was:', anonymized.y[i_test][:5])
    return mean_squared_error(anonymized.y[i_test], y_pred) ** 0.5

In [208]:
def run_simulation():
    kept_works = select_works_to_rate()
    i_train, i_test = split_train_test(kept_works)
    print('train size', len(i_train))
    print('test size', len(i_test))
    rmse = compute_error(i_train, i_test)
    print(NB_WORKS, METHOD, rmse)

## Results (number of works asked, strategy, RMSE)

- 10 DPP 1.09488816919 1.1020390692
- 20 DPP 1.0957652159900086
- 50 DPP 1.06436560838
- 100 DPP 1.0724727205
- 500 DPP 1.27818830679 0.848657862475 (possibly overfitting)

- 10 RND 1.09689215268 1.08273857127
- 20 RND 1.11930399192
- 50 RND 1.06455730002
- 100 RND 1.06357987938
- 500 RND 0.825381153284 0.754057572138

In [204]:
NB_WORKS = 500
METHOD = 'dpp'

In [205]:
run_simulation()

317844
37
[ 0.85630734  0.97026515  0.68152273  0.564       0.96290761]
[-0.5  0.5  0.5  0.5  2. ]
500 dpp 0.848657862475


Still needs to be done:
- run simulation on several users (how many works should be kept?)
- try a different number of neighbors
- should we evaluate the strategy on the whole profile or just the top recommendations?
- try other models than MangakiKNN (but others are slow)